In [1]:
#importing the libraries
import keras
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPooling2D
from keras.layers import Flatten
from keras.layers import Dense

In [2]:
classifier= Sequential() # Initialise the CNN

In [3]:
# Ist step of Convoltional layer to get feature maps using feature detector
classifier.add(Convolution2D(filters=32, # output feature maps
                             kernel_size=(3,3), # matrix size for feature detector
                             input_shape=(64, 64, 3), # input image shape, 3 is for rgb coloured image with 128*128 px
                             kernel_initializer='he_uniform', # weights distriution
                             activation='relu')) # activation function

In [4]:
# 2nd Pooling layer
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [5]:
#2nd convolutional and pooling layer.
classifier.add(Convolution2D(filters=32,
                             kernel_size=(3,3),
                             kernel_initializer='he_uniform',
                             activation='sigmoid'))
classifier.add(MaxPooling2D(pool_size=(2,2)))

In [6]:
# Step 3 - Flattening
classifier.add(Flatten())

In [7]:
#Step 4 full connection in which input we have from flattening

classifier.add(Dense(units=128,kernel_initializer='glorot_uniform', activation='relu'))
#step 5 output layer
classifier.add(Dense(units=1,kernel_initializer='glorot_uniform',activation='sigmoid'))
# Compiling the CNN
classifier.compile(optimizer = 'adam', loss = 'binary_crossentropy', metrics = ['accuracy'])

In [8]:
classifier.summary

<bound method Model.summary of <keras.src.engine.sequential.Sequential object at 0x000001FE74436090>>

In [9]:
from keras.preprocessing.image import ImageDataGenerator

In [10]:
#applying all the transformation we want to apply to training data set
train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

In [11]:
#Rescling the test data set images to use for validation.
test_datagen= ImageDataGenerator(rescale=1./255)

In [12]:
#Getting My training data ready for validation, so it will read all the data with the px size we gave.

training_set= train_datagen.flow_from_directory(directory= 'Dataset ASD/train',
                                               target_size=(64,64), # As we choose 64*64 for our convolution model
                                               batch_size=50,
                                               class_mode='binary' # for 2 class binary
                                               )

Found 254 images belonging to 2 classes.


In [15]:
#Getting My test data ready for validation, so it will read all the data with the px size we gave.

test_set= test_datagen.flow_from_directory(directory= 'Dataset ASD/val',
                                               target_size=(64,64), # As we choose 64*64 for our convolution model
                                               batch_size=50,
                                               class_mode='binary' # for 2 class binary
                                          )


Found 200 images belonging to 2 classes.


In [16]:
classifier.fit_generator(training_set, #training data to fit
                        #steps_per_epoch=12, # Data in training set
                        epochs= 200, # No of epochs to run
                        validation_data=test_set, # Test or validation set
                        #validation_steps=10 # no of data point for validation
                        )

C:\Users\Administrator\AppData\Local\Temp\ipykernel_7284\1355453033.py:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(training_set, #training data to fit


Epoch 1/200


6/6 [==============================] - 14s 1s/step - loss: 1.0450 - accuracy: 0.5118 - val_loss: 0.6932 - val_accuracy: 0.5000
Epoch 2/200
6/6 [==============================] - 2s 349ms/step - loss: 0.6901 - accuracy: 0.5906 - val_loss: 0.6949 - val_accuracy: 0.5000
Epoch 3/200
6/6 [==============================] - 2s 358ms/step - loss: 0.6789 - accuracy: 0.5906 - val_loss: 0.7059 - val_accuracy: 0.5000
Epoch 4/200
6/6 [==============================] - 2s 358ms/step - loss: 0.6786 - accuracy: 0.5906 - val_loss: 0.7316 - val_accuracy: 0.5000
Epoch 5/200
6/6 [==============================] - 2s 368ms/step - loss: 0.6836 - accuracy: 0.5906 - val_loss: 0.7373 - val_accuracy: 0.5000
Epoch 6/200
6/6 [==============================] - 2s 351ms/step - loss: 0.6791 - accuracy: 0.5906 - val_loss: 0.7118 - val_accuracy: 0.5000
Epoch 7/200
6/6 [==============================] - 2s 284ms/step - loss: 0.6738 - accuracy: 0.5906 - val_loss: 0.7619 - val_accuracy: 0.5000
Epoch 8/200
6

In [16]:
import numpy as np
from keras.preprocessing import image
#test_image = image.load_img('/content/drive/MyDrive/dataset/test/tdc/Leuven_2_0050722_lfcd.nii.gz.png', target_size = (64, 64))
test_image = image.load_img('Dataset ASD/val/asd/Leuven_2_0050747_lfcd.nii.gz.png', target_size = (64, 64))
# Loading the image and converting the pixels into array whcih will be used as input to predict.
test_image = image.img_to_array(test_image)
test_image = np.expand_dims(test_image, axis = 0)
result = classifier.predict(test_image)
training_set.class_indices
print(result[0][0])
if result[0][0]<0.5:
    prediction = 'asd'
else:
    prediction = 'tdc'
print(prediction)

1/1 [==============================] - 1s 984ms/step
0.7073768
tdc


In [20]:
classifier.save("200epochsModel.h5")

c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
